In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# INITIAL CLEAN OF THE Function Invocation Counts file of the Azure Function dataset

import pandas as pd

# Step 1: Load the dataset
file_path = '../Datasets/azurefunctions-dataset2019/invocations_per_function_md.anon.d01.csv'
df = pd.read_csv(file_path)

# Step 2: Exploration by displaying basic information and statistics about the dataset 
print("\nSummary Statistics:")
print(df.describe(include='all'))
print(df.info())
print(df.head())


Summary Statistics:
                                                HashOwner  \
count                                               46412   
unique                                              11039   
top     486e10ac87f2c7495ab672094483c49e935e6f7e19b2b9...   
freq                                                  791   
mean                                                  NaN   
std                                                   NaN   
min                                                   NaN   
25%                                                   NaN   
50%                                                   NaN   
75%                                                   NaN   
max                                                   NaN   

                                                  HashApp  \
count                                               46412   
unique                                              17577   
top     cb34fd874e255ddeaf1a38d86e7f3a41dffb0efdee8e46...   
fr

In [10]:
import pandas as pd

file_path = '../Datasets/azurefunctions-dataset2019/invocations_per_function_md.anon.d01.csv'
df = pd.read_csv(file_path)

# Step 3: Identify missing values
missing_values = df.isnull().sum()

# Step 4: Filter and sort columns with missing values
missing_values_summary = missing_values[missing_values > 0].sort_values(ascending=False)

# Step 5: Display the summary of missing values
print("Missing Values Summary:")
print(missing_values_summary)

Missing Values Summary:
Series([], dtype: int64)


In [9]:
# Extract the first 10 rows with all columns of the dataset
extracted_df = df.iloc[:10, :]

# Display the extracted dataset
print("\nExtracted Dataset First 10 Rows:")
print(extracted_df)

# Save the extracted dataset to a new CSV file
# extracted_df.to_csv('extract_invocations_per_function.csv', index=False)



Extracted Dataset First 10 Rows:
                                           HashOwner  \
0  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
1  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
2  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
3  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
4  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
5  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
6  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
7  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
8  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
9  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   

                                             HashApp  \
0  7ca324d9fc836a5d4562811c11ce3719530ee919dd1fb9...   
1  0d0ac65651f54ae3285a59564d64e39238b516fa1d5b56...   
2  a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5...   
3  a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5...   
4  a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5...   
5  a04487a6ba

In [18]:
import pandas as pd

# Load the data
file_path = '../Datasets/azurefunctions-dataset2019/invocations_per_function_md.anon.d01.csv'
df = pd.read_csv(file_path)

# Select invocation columns and convert them to integers
invocation_columns = [str(i) for i in range(1, 1441)]
data_hourly = df[invocation_columns].copy()
data_hourly.columns = range(1, 1441)  # Convert column names to integers for grouping

# Transpose, group by every 60 columns (for hourly aggregation), then transpose back
data_hourly = data_hourly.T.groupby(lambda x: (x - 1) // 60).sum().T

# Rename columns to hourly format (0 to 23)
data_hourly.columns = [f'Hour_{i}' for i in range(24)]

# Combine with original identifiers and trigger type
data_processed = pd.concat([df[['HashOwner', 'HashApp', 'HashFunction', 'Trigger']], data_hourly], axis=1)

# Identify peak hours for each function (e.g., top 3 peak hours)
peak_hours = data_hourly.apply(lambda x: x.nlargest(3).index.tolist(), axis=1)  # Top 3 peak hours

# Add peak hours as a new column
data_processed['Peak_Hours'] = peak_hours
print(data_processed.head())

# Save processed data
data_processed.to_csv('../Processed/processed_invocations_per_function.csv', index=False)

                                           HashOwner  \
0  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
1  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
2  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
3  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   
4  71ca12c7af70d021e285b51b245942f8432df6463ff9f2...   

                                             HashApp  \
0  7ca324d9fc836a5d4562811c11ce3719530ee919dd1fb9...   
1  0d0ac65651f54ae3285a59564d64e39238b516fa1d5b56...   
2  a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5...   
3  a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5...   
4  a04487a6ba1e14296eb7647e4963180d28bef7a90a8fc5...   

                                        HashFunction        Trigger  Hour_0  \
0  520dbd6bd906840012aa0c4b778743efc7c0ac7b7caf96...           http      12   
1  115ca7a2b5bc290052c3da74cd0347d19c3c67b7d5aa66...           http      12   
2  93e6c664773bbec3a7f50a0e92fa7e97401a802dc6eed8...  orchestration       0   
3  740c5c7